# Machine Learning - Tarea 2
---

---

<div style="text-align: right">
Vicente Lizana - 201310004-K
</div>
<div style="text-align: right">
Álvaro Valderrama - 201390005-4
</div>

---

---

---
## 1.- 

---
### 1.a

#### Placeholder

---
### 1.b

#### Placeholder

---
### 1.c

#### Placeholder

---
### 1.d

#### Placeholder


---
### 1.e

#### Placeholder


---
### 1.f

#### Placeholder

---
### 1.g

#### Placeholder

---
### 1.h

#### Placeholder

---
### 1.i

#### Placeholder

---

---
## 2.- 

---
### 2.a

#### Placeholder

---
### 2.b

#### Placeholder

---
### 2.c

#### Placeholder

---
### 2.d

#### Placeholder

---
### 2.e

#### Placeholder

---
### 2.f

#### Placeholder

---
### 2.g

#### Placeholder

---

---
## 3.- 

---
### 3.a

#### Placeholder

---
### 3.b

#### Placeholder

---
### 3.c

#### Placeholder

---
### 3.d

#### Placeholder

---
### 3.e

#### Placeholder

---
### 3.f

#### Placeholder

---
### 3.g

#### Placeholder

---

---

---